In [ ]:
import pandas
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
electricity_data = pandas.read_csv('/home/ctang/ttmp/Datathon/data_and_starter_code/data/electric_prices (1).csv')
energy_data = pandas.read_csv('/home/ctang/ttmp/Datathon/data_and_starter_code/data/baltic_energy_type.csv')
carbon_data = pandas.read_csv('/home/ctang/ttmp/Datathon/data_and_starter_code/data/carbon_intensity_countries.csv')

In [ ]:
# join carbon data with electricity data on date
carbon_data['Datetime (UTC)'] = pandas.to_datetime(carbon_data['Datetime (UTC)'])
electricity_data['Date'] = pandas.to_datetime(electricity_data['Date'])
carbon_data = carbon_data.rename(columns={'Datetime (UTC)': 'Date'})
carbon_data = carbon_data.set_index('Date')
carbon_data = carbon_data.drop(columns=['Zone id', 'Data source'])
electricity_data = electricity_data.set_index('Date')

In [ ]:
carbon_data_wide = carbon_data.pivot_table(index=carbon_data.index, columns='Country')
carbon_data_wide.columns = ['{}_{}'.format(col[1], col[0]) for col in carbon_data_wide.columns]
carbon_data = carbon_data_wide

In [ ]:
# join electricity data with carbon data on date
combined_data = electricity_data.join(carbon_data, how='left')
combined_data.columns

In [ ]:
# plot electricity price on y vs carbon intensity on x for all countries, plot separate plots for each year. use percent carbon-free energy as hue
combined_data['Year'] = combined_data.index.year
combined_data['Date'] = combined_data.index

# compute log of carbon free energy percentage in new column called log_Carbon-free energy percentage (CFE%)
combined_data['Estonia_Carbon-free energy percentage (CFE%)'] = combined_data['Estonia_Carbon-free energy percentage (CFE%)'].replace(0, 0.0001)
combined_data['Latvia_Carbon-free energy percentage (CFE%)'] = combined_data['Latvia_Carbon-free energy percentage (CFE%)'].replace(0, 0.0001)
combined_data['Lithuania_Carbon-free energy percentage (CFE%)'] = combined_data['Lithuania_Carbon-free energy percentage (CFE%)'].replace(0, 0.0001)

fig, ax = plt.subplots(figsize = (40,20))
combined_data.plot.scatter(x='Date', s='Estonia', ax = ax, y='Estonia_log_Carbon-free energy percentage (CFE%)', c = 'blue', label = 'Estonia', alpha = 0.5)
combined_data.plot.scatter(x='Date', s='Latvia', ax = ax, y='Latvia_log_Carbon-free energy percentage (CFE%)', c = 'green', label = 'Latvia', alpha = 0.5)
combined_data.plot.scatter(x='Date', s='Lithuania', ax = ax, y='Lithuania_log_Carbon-free energy percentage (CFE%)', c = 'red', label = 'Lithuania', alpha = 0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Electricity Price (EUR/MWh)')
ax.set_title('Latvia electricity price vs carbon intensity')
plt.show()

In [ ]:
for year in combined_data['Year'].unique():
    # only plot if there is more than 10 data points
    if len(combined_data[combined_data['Year'] == year]) > 10:
        fig, ax = plt.subplots(figsize=(15, 5))
        combined_data.plot.scatter(x='Date', y='Latvia_Carbon-free energy percentage (CFE%)', ax = ax, c='Latvia', colormap='viridis')
        ax.set_xlabel('Electricity price (EUR/MWh)')
        ax.set_ylabel('Carbon intensity (gCO2/kWh)')
        ax.set_title('Latvia electricity price vs carbon intensity in {}'.format(year))
        plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
combined_data.plot.scatter(x='Date', y='Latvia_Carbon-free energy percentage (CFE%)', ax = ax, c='Estonia_Carbon intensity gCO‚ÇÇeq/kWh (Life cycle)', colormap='viridis')
ax.set_xlabel('Electricity price (EUR/MWh)')
ax.set_ylabel('Carbon intensity (gCO2/kWh)')
ax.set_title('Latvia electricity price vs carbon intensity')
plt.show()

In [ ]:
# scatter plot of Estonian carbon-free energy percentage vs time, smoothed over 30 days, set color bar name to Carbon Intensity (gCO2/kWh)
plt.figure(figsize=(15, 5))
combined_data['Estoan_smooth'] = combined_data['Estonia_Carbon-free energy percentage (CFE%)'].rolling(window=30).mean()
scatter = plt.scatter(x=combined_data['Date'], y=combined_data['Estoan_smooth'], c=combined_data['Estonia_Carbon intensity gCO‚ÇÇeq/kWh (Life cycle)'])
cbar = plt.colorbar(scatter)
cbar.set_label('Lifetime Carbon Intensity (gCO2/kWh)')
plt.xlabel('Date')
plt.ylabel('Estonian carbon-free energy percentage')
plt.title('Estonian carbon-free energy percentage vs time')
plt.show()